In [ ]:
# Importiamo le librerie

from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
import pickle

In [ ]:
with open('pickle/Friday-16-02-2018_TrafficForML_CICFlowMeter/scaled_document.pickle', 'rb') as handle:

    df = pickle.load(handle)

In [ ]:
display(df)

In [ ]:
df_prova = df.groupby(['Label'])['Label'].count()


df_prova=df_prova.to_frame()
print(df_prova)
df_prova.set_index('Label')
df_prova=df_prova.rename(columns={'Label':'Count'})

In [ ]:
df_prova['test'] = np.random.randint(1, 10, df_prova.shape[0])

In [ ]:
display(df_prova)

In [ ]:
nome_file='Friday-02-03-2018_TrafficForML_CICFlowMeter'

In [ ]:
lista_comb=pd.read_csv('top_feature/'+nome_file+'/combinazioni.csv')

In [ ]:
# take only annual income and spending score
from sklearn.cluster import KMeans
for index,row in lista_comb.iterrows():
    a = pd.DataFrame(data={row[0]:df[row[0]],row[1]:df[row[1]]})
    x = a.iloc[:, [0, 1]].values

    # let's check the shape of x
    print(x.shape)

    km = KMeans(n_clusters = 2, init = 'random', max_iter = 100, n_init = 5, random_state = 0)
    y_means = km.fit_predict(x)

    plt.figure(figsize=(15,8))
    plt.scatter(x[y_means == 0, 0], x[y_means == 0, 1], s = 100, c = 'magenta', label = 'Benign')
    plt.scatter(x[y_means == 1, 0], x[y_means == 1, 1], s = 100, c = 'pink', label = 'Bot')
    plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centroid')
    plt.title('K Means Clustering', fontsize = 20)
    plt.xlabel(row[0])
    plt.ylabel(row[1])
    plt.legend()
    plt.show()
    title = str(row[0])+' - '+str(row[1])
    feature_name = title.replace('/','_')
    if not os.path.exists('image/'+nome_file+'/cluster_KMeans/'):
        os.makedirs('image/'+nome_file+'/cluster_KMeans/')
    plt.savefig('image/'+nome_file+'/cluster_KMeans/'+str(feature_name)+'.png')

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random

prova = df.values
neigh = NearestNeighbors(n_neighbors=5, n_jobs=4, algorithm='ball_tree', leaf_size=5)
nbrs = neigh.fit(prova)
distances, indices = nbrs.kneighbors(prova)

distances = np.sort(distances, axis=0)
plt.figure(figsize=(12,8))
plt.plot(distances[:,1])

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=1, min_samples=6).fit(prova)
ymeans = db.labels_
ymeans

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from minisom import MiniSom

# Genera dati di esempio
X, _ = make_blobs(n_samples=100, n_features=2, centers=4, random_state=0)

# Definisci le dimensioni della mappa
map_width = 10
map_height = 10

# Inizializza la SOM
som = MiniSom(map_width, map_height, 2, sigma=1.0, learning_rate=0.5)

# Inizializza i pesi in modo casuale
som.random_weights_init(X)

# Addestra la SOM
som.train_batch(X, 1000)

# Trova il BMU (Best Matching Unit) per ogni campione di input
bmu_indexes = np.array([som.winner(x) for x in X])

# Stampa i BMU e i relativi vettori di peso
for i, (x, bmu) in enumerate(zip(X, bmu_indexes)):
    print(f"Campione {i+1}:")
    print("Input:", x)
    print("BMU Indice:", bmu)
    print("BMU Peso:", som.weights[bmu[0], bmu[1]])
    print()
